In [1]:
from mnist_class import MNISTDataModule, FeedForwardNet, LogPredictionsCallback
import torchmetrics as tm
import pytorch_lightning as pl
from omegaconf import OmegaConf
import common
from haikunator import Haikunator
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import wandb
import torch as t

In [2]:
hparams = OmegaConf.create({
    "n_layer_1": 128,
    "n_layer_2": 256,
    "batch_size": 256,
    "lr": 0.05,
    "n_epochs": 3
})

In [3]:
project = "MnistNotebookDgb"
run_name = Haikunator().haikunate()
print(project, run_name)

MnistNotebookDgb square-dream-6374


In [4]:
model = FeedForwardNet(hparams, n_classes=10)

In [5]:
mnist = MNISTDataModule(hparams)

In [6]:
logger = WandbLogger(
    project=project,
    name=run_name,
    log_model="all",
    id=run_name
)
logger.watch(model, log="all")

wandb: Currently logged in as: avilay (use `wandb login --relogin` to force relogin)


In [7]:
checkpoint = ModelCheckpoint(monitor="val_loss", mode="min")
callbacks = [checkpoint, LogPredictionsCallback()]
trainer = pl.Trainer(
    max_epochs=hparams.n_epochs,
    logger=logger,
    callbacks=callbacks,
    progress_bar_refresh_rate=0
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
trainer.fit(model, mnist)


  | Name    | Type             | Params
---------------------------------------------
0 | fc1     | Linear           | 100 K 
1 | fc2     | Linear           | 33.0 K
2 | fc3     | Linear           | 2.6 K 
3 | loss_fn | CrossEntropyLoss | 0     
---------------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


APTG DEBUG: Inside callback
APTG DEBUG: Inside callback
APTG DEBUG: Inside callback
APTG DEBUG: Inside callback


In [9]:
wandb.finish()

_runtime,31
_timestamp,1625363378
_step,21
train_loss_step,0.68477
epoch,2
trainer/global_step,644
val_loss,0.70176
val_acc,0.7346
train_loss,0.68143
train_acc,0.72873


_runtime,▁▂▂▃▃▃▃▃▄▄▅▅▅▅▆▆▆▇▇███
_timestamp,▁▂▂▃▃▃▃▃▄▄▅▅▅▅▆▆▆▇▇███
_step,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
train_loss_step,▇▄▃█▄▄▁▂▁▂▂▂
epoch,▁▁▁▁▁▁▅▅▅▅▅▅██████
trainer/global_step,▁▂▂▃▃▃▃▃▄▅▅▅▅▅▆▆▇▇▇██
val_loss,█▂▁
val_acc,▁▆█
train_loss,█▂▁
train_acc,▁▆█
